In [1]:
import sys
sys.path.insert(0, '../')

from interface import get_query_emb_batch, load_model, cosine_similarity_batch
from typing import Tuple, List, Dict, Union, Any
from colbert.modeling.checkpoint import Checkpoint

from sklearn.metrics import precision_score, recall_score, accuracy_score

from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch

# pth_models = "/home/sondors/Documents/price/BERT_data/data/10-04-2024_Timofey/2801_models_Apple.csv"
pth_models = "/home/sondors/Documents/price/BERT_data/data/17-04-2024_Timofey/2801_offers_models_Apple.csv"
df_models = pd.read_csv(pth_models, sep=';')

pth_offers = '/home/sondors/Documents/price/BERT_data/data/17-04-2024_Timofey/2801_Apple_triplets_offer_model_train.csv'
df_offers = pd.read_csv(pth_offers, sep=';')

pth_offers_test = '/home/sondors/Documents/price/BERT_data/data/17-04-2024_Timofey/2801_offers_test_Apple.csv'
df_offers_test = pd.read_csv(pth_offers, sep=';')

id_category = {
    # 3902: 'диктофоны, портативные рекордеры',
    # 510402: 'электронные книги',
    # 4302: 'автомобильные телевизоры, мониторы',
    # 2815: 'смарт-часы и браслеты',
    # 3901: 'портативные медиаплееры',
    # 3904: 'портативная акустика',
    2801: 'мобильные телефоны',
    # 3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
    # 510401: 'планшетные компьютеры и мини-планшеты',
    # 2102: 'наушники, гарнитуры, наушники c микрофоном',
    # 3903: 'радиоприемники, радиобудильники, радиочасы',
    # 3907: 'магнитолы',
    # 280801: 'GPS-навигаторы'
    }

df_models = df_models[df_models['category_id'].isin(id_category.keys())].reset_index(drop=True)
df_offers = df_offers[df_offers['category_id'].isin(id_category.keys())].reset_index(drop=True)

df_offers

,name,model_id,offer_id,modelized_by_barcode,category_name,category_id,price,attrs,description,external_category,external_brand,external_type,true_match,false_match
0,Смартфоны Смартфон Apple iPhone 12 A2403 128ГБ...,7883241,1530924893,True,мобильные телефоны,2801,53060.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","Смартфон Apple A2403 iPhone 12 черный, процесс...",Смартфоны,APPLE,Смартфоны,"Apple iPhone 12 128Gb, nano-Sim+eSIM, Black","Apple iPhone 12 128Gb, eSIM, Green"
1,Apple iPhone 14 Pro 128ГБ Silver (Серебристый)...,7883526,1626555801,True,мобильные телефоны,2801,118050.0,"[{'id': 14519, 'type': 'b', 'value': 1.0}, {'i...",Apple iPhone 14 Pro — супербыстрый смартфон с ...,Телефоны :: Телефоны Apple iPhone,Apple,NaN,"Apple iPhone 14 Pro 128Gb, Dual nano-SIM, Silver","Apple iPhone 14 Pro 128Gb, Dual nano-SIM, Gold"
2,Смартфон Apple iPhone 12 256Gb Blue (А2403),7883255,1690046919,False,мобильные телефоны,2801,77499.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Смартфон Apple iPhone 12 256Gb Blue (А2403),Смартфоны и телефоны :: Apple :: iPhone 12,NaN,NaN,"Apple iPhone 12 256Gb, nano-Sim+eSIM, Синий","Apple iPhone 12 256Gb, nano-Sim+eSIM, Зеленый"
3,Смартфон Apple iPhone 12 128GB MGJA3HN/A Black,7883241,1540988497,True,мобильные телефоны,2801,62990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","iPhone 12. Во-первых, это быстро. iPhone 12 по...",Телефоны :: Смартфоны,Apple,Смартфон,"Apple iPhone 12 128Gb, nano-Sim+eSIM, Black","Apple iPhone 12 128Gb, eSIM, Green"
4,"Смартфон Apple iPhone 14 Pro 1 ТБ, Dual nano S...",7883536,1837804366,True,мобильные телефоны,2801,151170.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Новый волшебный способ взаимодействия с iPhone...,Все товары :: Электроника :: Телефоны :: Мобил...,Apple,Смартфон,"Apple iPhone 14 Pro 1Tb, Dual nano-SIM, Deep P...","Apple iPhone 14 Pro 1Tb, Dual eSIM, Space Black"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42715,Смартфон Apple iPhone 15 Plus 128GB eSIM Black...,7883661,1810174582,False,мобильные телефоны,2801,80490.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Возможна доставка в день заказа. Квалифицирова...,Телефоны :: iPhone 15 Plus :: iPhone 15 Plus 1...,Apple,Смартфон,"Apple iPhone 15 Plus 128Gb, Dual eSIM, Black","Apple iPhone 15 512Gb, nano-Sim+eSIM, Черный"
42716,Смартфон Apple iPhone 14 Plus 128Gb Purple (фи...,7883483,1844276843,False,мобильные телефоны,2801,76090.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Возможна доставка в день заказа. Квалифицирова...,Телефоны :: iPhone 14 Plus :: iPhone 14 Plus 1...,Apple,Смартфон,"Apple iPhone 14 Plus 128Gb, nano-Sim+eSIM, Фио...","Apple iPhone 14 Plus 128Gb, nano-Sim+eSIM, Тём..."
42717,Смартфон Apple iPhone 14 Plus 256Gb Purple (фи...,7883496,1844276848,False,мобильные телефоны,2801,88990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Возможна доставка в день заказа. Квалифицирова...,Телефоны :: iPhone 14 Plus :: iPhone 14 Plus 2...,Apple,Смартфон,"Apple iPhone 14 Plus 256Gb, nano-Sim+eSIM, Purple","Apple iPhone 14 Plus 256Gb, Dual nano-SIM, Черный"
42718,Смартфон Apple iPhone 14 Plus 512Gb Blue (голу...,7883515,1851901283,False,мобильные телефоны,2801,93990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Возможна доставка в день заказа. Квалифицирова...,Телефоны :: iPhone 14 Plus :: iPhone 14 Plus 5...,Apple,Смартфон,"Apple iPhone 14 Plus 512Gb, nano-Sim+eSIM, Blue","Apple iPhone 14 Plus 512Gb, nano-Sim+eSIM, (PR..."


In [2]:
# ckpt_pth = "/home/sondors/Documents/ColBERT_weights/triples_X1_13_categories_use_ib_negatives/none/2024-01/26/10.49.44/checkpoints/colbert-5387-finish"
# experiment = "colbert-5387"

ckpt_pth = "/home/sondors/Documents/ColBERT_weights/2801_lr04_bsize_210_apple/none/2024-04/18/09.16.10/checkpoints/colbert-187-finish"
experiment = "colbert-187-finish"

doc_maxlen = 300
nbits = 2   # bits определяет количество битов у каждого измерения в семантическом пространстве во время индексации
nranks = 1  # nranks определяет количество GPU для использования, если они доступны
kmeans_niters = 4 # kmeans_niters указывает количество итераций k-means кластеризации; 4 — хороший и быстрый вариант по умолчанию. 

device = "cuda"


In [3]:
checkpoint = load_model(ckpt_pth, doc_maxlen, nbits, kmeans_niters, device)

In [4]:
def get_query_emb(sentences: List[str], checkpoint: Checkpoint, batch_size: int) -> np.ndarray:
    """
    Generate embeddings for a list of sentences using the provided checkpoint.

    Args:
        sentences (List[str]): A list of sentences for which embeddings need to be generated.
        checkpoint (Checkpoint): The checkpoint object used for generating embeddings.
        batch_size (int): The batch size to use during inference.

    Returns:
        np.ndarray: An array of embeddings for the input sentences.
    """
    return checkpoint.queryFromText(sentences, bsize=batch_size).to("cpu").numpy()

def get_query_emb_batch(sentences: List[str], checkpoint: Checkpoint, batch_size: int, batch_size2: int) -> np.ndarray:
    """
    Generate embeddings for a list of sentences in batches using the provided checkpoint.

    Args:
        sentences (List[str]): A list of sentences for which embeddings need to be generated.
        checkpoint (Checkpoint): The checkpoint object used for generating embeddings.
        batch_size (int): The batch size to use during inference.
        batch_size2 (int): The size of the sub-batches to split the input sentences into.

    Returns:
        np.ndarray: An array of embeddings for the input sentences. 
        Shape of the array is (len(sentences), 32, 768) for bert-base-multilingual-cased or (len(sentences), 32, 128) for colbertv2.0
    """
    embeddings_list = []
    
    for i in range(0, len(sentences), batch_size2):
        # print(f"batch: {min(i+batch_size2, len(sentences))}/{len(sentences)}")

        batch_sentences = sentences[i:i+batch_size2]
        embeddings = get_query_emb(batch_sentences, checkpoint, batch_size)
        embeddings_list.append(embeddings)

        torch.cuda.empty_cache()
    
    combined_embeddings = np.concatenate(embeddings_list, axis=0)
    return combined_embeddings

In [5]:

df_offers_shuffled = df_offers.sample(frac=1, random_state=42)

# Определяем размер тестовой выборки (например, 20%)
test_size = int(0.15 * len(df_offers_shuffled))

# Разделяем данные на тренировочную и тестовую выборки
df_train = df_offers_shuffled.iloc[:-test_size]
df_test = df_offers_shuffled.iloc[-test_size:]

# Пример: вывод размеров тренировочной и тестовой выборок
print("Размер тренировочной выборки:", len(df_train))
print("Размер тестовой выборки:", len(df_test))


Размер тренировочной выборки: 36312
Размер тестовой выборки: 6408


In [6]:

def batch_generator_from_df(df, batch_size):
    num_samples = len(df)
    num_batches = num_samples // batch_size
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        
        offer_batch = df['name'].iloc[start_idx:end_idx].values
        true_match_batch = df['true_match'].iloc[start_idx:end_idx].values
        false_match_batch = df['false_match'].iloc[start_idx:end_idx].values
        
        yield list(offer_batch), list(true_match_batch), list(false_match_batch)

def prepare_X_y1(offer_batch, true_match_batch, false_match_batch, checkpoint):

    offer_embs = get_query_emb_batch(offer_batch, checkpoint, batch_size=1000, batch_size2=10000)
    true_match_embs = get_query_emb_batch(true_match_batch, checkpoint, batch_size=1000, batch_size2=10000)
    false_match_embs = get_query_emb_batch(false_match_batch, checkpoint, batch_size=1000, batch_size2=10000)

    y_true = np.ones(len(true_match_embs))  # Метка 1 для пар (offer_embs[i], true_match_embs[i])
    y_false = np.zeros(len(false_match_embs))  # Метка 0 для пар (offer_embs[i], false_match_embs[i])
    
    X_true = np.concatenate([offer_embs, true_match_embs], axis=1)
    X_false = np.concatenate([offer_embs, false_match_embs], axis=1)

    del offer_embs, true_match_embs, false_match_embs
    torch.cuda.empty_cache()

    X = np.concatenate([X_true, X_false])
    y = np.concatenate([y_true, y_false])

    # Перемешиваем данные
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    return X, y

def prepare_X_y(offer_batch, true_match_batch, false_match_batch, checkpoint):

    offer_embs = get_query_emb_batch(offer_batch, checkpoint, batch_size=1000, batch_size2=10000)
    true_match_embs = get_query_emb_batch(true_match_batch, checkpoint, batch_size=1000, batch_size2=10000)
    false_match_embs = get_query_emb_batch(false_match_batch, checkpoint, batch_size=1000, batch_size2=10000)

    y_true = np.ones(len(true_match_embs))  # Метка 1 для пар (offer_embs[i], true_match_embs[i])
    y_false = np.zeros(len(false_match_embs))  # Метка 0 для пар (offer_embs[i], false_match_embs[i])
    
    # Создаем обучающий набор данных
    X_true = np.concatenate([offer_embs, true_match_embs], axis=1)
    X_false = np.concatenate([offer_embs, false_match_embs], axis=1)

    del offer_embs, true_match_embs, false_match_embs
    torch.cuda.empty_cache()

    X = np.concatenate([X_true, X_false])
    y = np.concatenate([y_true, y_false])

    # Перемешиваем данные
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    return X.reshape((X.shape[0], -1)), y

# N = 10  # Размер батча
df = df_train[:10000].copy()
# batch_gen = batch_generator_from_df(df, N)
# for offer_batch, true_match_batch, false_match_batch in batch_gen:
#     # print("Offer batch:", offer_batch)
#     # print("True match batch:", true_match_batch)
#     # print("False match batch:", false_match_batch)

#     X, y = prepare_X_y(offer_batch, true_match_batch, false_match_batch, checkpoint)

offer_batch = list(df['name'])
true_match_batch = list(df['true_match'])
false_match_batch = list(df['false_match'])

X, y = prepare_X_y(offer_batch, true_match_batch, false_match_batch, checkpoint)

print(np.shape(X))
print(np.shape(y))

(20000, 49152)
(20000,)


In [7]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

def train_svm(X_train, y_train):
    # Создаем конвейер, который включает в себя стандартизацию данных и SVM с RBF ядром
    svm_model = make_pipeline(StandardScaler(), SVC(kernel='rbf', probability=True))
    svm_model.fit(X_train, y_train)
    return svm_model

def train_svm_batched(X_train, y_train, batch_size=1000):
    # Перемешиваем данные
    num_samples = X_train.shape[0]
    num_batches = (num_samples + batch_size - 1) // batch_size

    # Создаем конвейер, который включает в себя стандартизацию данных и SVM с RBF ядром
    svm_model = make_pipeline(StandardScaler(), SVC(kernel='rbf', probability=True))

    # Обучаем модель на каждом батче
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, num_samples)
        X_batch = X_train[start_idx:end_idx]
        y_batch = y_train[start_idx:end_idx]
        svm_model.fit(X_batch, y_batch)  # Обучение на каждом батче

    return svm_model
# Допустим, у вас уже есть данные X_train и y_train, которые вы получили с помощью функции prepare_X_y
# X_train размерность (N, 1536) - 64*768=49152
# y_train размерность (N,)

# Обучаем SVM
svm_classifier = train_svm_batched(X, y)


In [ ]:
df = df_test[:100].copy()

offer_batch = list(df['name'])
true_match_batch = list(df['true_match'])
false_match_batch = list(df['false_match'])

X_test, y_test = prepare_X_y(offer_batch, true_match_batch, false_match_batch, checkpoint)


In [ ]:

probabilities = svm_classifier.predict_proba(X_test)[:, 1]  # Вероятности класса 1

# Нормализуем вероятности к диапазону от 0 до 1
min_prob = min(probabilities)
max_prob = max(probabilities)
normalized_probs = (probabilities - min_prob) / (max_prob - min_prob)

[9.95517016e-01 9.20009771e-03 7.19760627e-03 9.95870731e-01
 8.75864878e-03 9.85554269e-01 1.25356099e-02 2.85740411e-01
 2.50357230e-03 9.78560747e-01 9.64420919e-03 2.86438464e-01
 2.64641416e-03 9.91561265e-01 9.62246566e-01 5.34998909e-03
 1.71737146e-02 9.28542283e-01 9.95189281e-01 9.62733191e-01
 8.00168330e-03 9.95241092e-01 9.97033468e-01 8.95525738e-01
 3.21311913e-03 1.02331491e-02 5.44383790e-02 5.89824844e-03
 6.66619194e-01 7.55711747e-01 8.87440514e-02 7.98795764e-03
 9.94956287e-01 9.99595677e-01 1.19968741e-03 9.97464318e-01
 7.74077634e-02 6.41709682e-03 4.23549098e-03 2.95798343e-02
 9.13927709e-01 6.41742073e-01 9.52009030e-01 9.94150316e-01
 9.97207343e-01 1.46422679e-01 1.31037258e-02 1.68428103e-02
 0.00000000e+00 9.96003501e-01 3.53067734e-03 8.51754504e-03
 1.26359855e-02 9.90304932e-01 9.93343806e-01 4.29056115e-02
 1.12315835e-03 7.20098961e-01 9.44212657e-01 9.68039916e-01
 9.89528110e-01 9.98631286e-01 9.67469576e-01 9.35359588e-02
 1.11617187e-02 9.912741

In [ ]:
best_preds = np.asarray([np.argmax(line) for line in probabilities])
accuracy = accuracy_score(y_test, best_preds)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 50.00%


### GPU

In [11]:
import cudf
from cuml.svm import SVC as cuSVC
from cuml.preprocessing import StandardScaler
from cuml.pipeline import make_pipeline

def train_svm_gpu(X_train, y_train):
    # Преобразуем данные в cuDF DataFrame
    X_train_cudf = cudf.DataFrame(X_train)
    y_train_cudf = cudf.Series(y_train)

    # Создаем конвейер, который включает в себя стандартизацию данных и SVM с RBF ядром
    svm_model = make_pipeline(StandardScaler(), cuSVC(kernel='rbf', probability=True))
    svm_model.fit(X_train_cudf, y_train_cudf)
    return svm_model

# Допустим, у вас уже есть данные X_train и y_train, которые вы получили с помощью функции prepare_X_y
# X_train размерность (N, 1536) - 64*768=49152
# y_train размерность (N,)

# Обучаем SVM на GPU
# svm_classifier_gpu = train_svm_gpu(X_train, y_train)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 import cudf                                                                                 │
│    2 from cuml.svm import SVC as cuSVC                                                           │
│    3 from cuml.preprocessing import StandardScaler                                               │
│    4 from cuml.pipeline import make_pipeline                                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'cudf'

In [ ]:

# Преобразуем тестовые данные в cuDF DataFrame, если они не были предварительно преобразованы
# X_test_cudf = cudf.DataFrame(X_test)

# Получаем вероятности принадлежности к классу 1 (match)
probabilities_gpu = svm_classifier_gpu.predict_proba(X_test)[:, 1]  # Вероятности класса 1

# Нормализуем вероятности к диапазону от 0 до 1
min_prob_gpu = min(probabilities_gpu)
max_prob_gpu = max(probabilities_gpu)
normalized_probs_gpu = (probabilities_gpu - min_prob_gpu) / (max_prob_gpu - min_prob_gpu)

print(normalized_probs_gpu)

In [ ]:
"""
def prepare_X_y(offer_batch, true_match_batch, false_match_batch, checkpoint):

    offer_embs = get_query_emb_batch(offer_batch, checkpoint, batch_size=1000, batch_size2=10000)
    true_match_embs = get_query_emb_batch(true_match_batch, checkpoint, batch_size=1000, batch_size2=10000)
    false_match_embs = get_query_emb_batch(false_match_batch, checkpoint, batch_size=1000, batch_size2=10000)

    y_true = np.ones(len(true_match_embs))  # Метка 1 для пар (offer_embs[i], true_match_embs[i])
    y_false = np.zeros(len(false_match_embs))  # Метка 0 для пар (offer_embs[i], false_match_embs[i])
    
    X_true = np.concatenate([offer_embs, true_match_embs], axis=1)
    X_false = np.concatenate([offer_embs, false_match_embs], axis=1)

    del offer_embs, true_match_embs, false_match_embs
    torch.cuda.empty_cache()

    X = np.concatenate([X_true, X_false])
    y = np.concatenate([y_true, y_false])

    # Перемешиваем данные
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    return X, y

Напиши код аналога косинусного расстояния через SVM. На вход поступают данные 
X размерность (N, 64, 768)
Y размерность (N,)
где N - количество обучающих примеров
X - конкатенация пар эмбеддингов матч и не матч
Y - Метка 1 или 0 для пар матч или не матч

На выходе из модели SVM должно быть число от 0 до 1
"""

In [14]:
print(np.shape(X))

(20, 49152)
